In [2]:
!pip install snakeviz
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [ ]:
!python --version

# cProfile metrics

---

- __ncalls:__ Shows the number of calls made
- __tottime:__ Total time taken by the given function. Note that the time made in calls to sub-functions are excluded.
- __percall:__ Total time / No of calls. ( remainder is left out )
- __cumtime:__ Unlike tottime, this includes time spent in this and all subfunctions that the higher-level function calls. It is most useful and is accurate for recursive functions.
- The percall following cumtime is calculated as the quotient of cumtime divided by primitive calls. The primitive calls include all the calls that were not included through recursion.

---



In [1]:
import os
import sys
import pandas as pd
import cProfile
import pstats
import time

from pyjedai.datamodel import Data
data = Data(
    dataset_1=pd.read_csv("./data/der/synthetic/D1-10K/X.csv", sep='|', engine='python', na_filter=False).astype(str),
    attributes_1=['Id', 'given_name', 'surname', 'date_of_birth', 'address_1',
       'address_2', 'street_number', 'postcode', 'soc_sec_id', 'suburb',
       'phone_number', 'state', 'age'],
    id_column_name_1='Id',
    ground_truth=pd.read_csv("./data/der/synthetic/D1-10K/GT.csv", sep='|', engine='python'),
)

from pyjedai.workflow import WorkFlow, compare_workflows
from pyjedai.block_building import StandardBlocking, QGramsBlocking, ExtendedQGramsBlocking, SuffixArraysBlocking, ExtendedSuffixArraysBlocking
from pyjedai.block_cleaning import BlockFiltering, BlockPurging
from pyjedai.comparison_cleaning import WeightedEdgePruning, WeightedNodePruning, CardinalityEdgePruning, CardinalityNodePruning, BLAST, ReciprocalCardinalityNodePruning, ReciprocalWeightedNodePruning, ComparisonPropagation
from pyjedai.matching import EntityMatching
from pyjedai.clustering import ConnectedComponentsClustering
from pyjedai.block_building import (
    StandardBlocking,
    QGramsBlocking,
    ExtendedQGramsBlocking,
    SuffixArraysBlocking,
    ExtendedSuffixArraysBlocking,
)

# %reload_ext snakeviz
# %snakeviz 

from pyjedai.vector_based_blocking import EmbeddingsNNBlockBuilding

# t1 = time.process_time()
# %reload_ext snakeviz
# %snakeviz  
qgb = StandardBlocking()
blocks = qgb.build_blocks(data, attributes_1=['Id', 'given_name', 'surname', 'date_of_birth', 'address_1', 'address_2', 'street_number', 'postcode', 'soc_sec_id', 'suburb', 'phone_number', 'state', 'age'])
# t2 = time.process_time() - t1
# print(t2)

bf = BlockFiltering(ratio=0.8)
filtered_blocks = bf.process(blocks, data, tqdm_disable=False)
# cbbp = BlockPurging()
# cleaned_blocks = cbbp.process(filtered_blocks, data, tqdm_disable=False)
wep = CardinalityEdgePruning(weighting_scheme='EJS')
%reload_ext snakeviz
%snakeviz candidate_pairs_blocks = wep.process(filtered_blocks, data, tqdm_disable=False)

# EM = EntityMatching()

# pairs_graph = EM.predict(candidate_pairs_blocks, data, tqdm_disable=True)
# ccc = ConnectedComponentsClustering()
# clusters = ccc.process(pairs_graph)

Standard Blocking:   0%|          | 0/10000 [00:00<?, ?it/s]

Block Filtering:   0%|          | 0/3 [00:00<?, ?it/s]

Cardinality Edge Pruning:   0%|          | 0/10000 [00:00<?, ?it/s]

 
*** Profile stats marshalled to file 'C:\\Users\\nikol\\AppData\\Local\\Temp\\tmp4jwby769'. 
Embedding SnakeViz in this document...


In [ ]:
from pyjedai.utils import print_blocks
print_blocks(qgb.blocks, data.is_dirty_er)

In [39]:
from pyjedai.evaluation import Evaluation, write
Evaluation(data).report(candidate_pairs_blocks)

#  Evaluation 
---
Scores:
	Precision:     17.31% 
	Recall:        96.24%
	F1-score:      29.35%
Classification report:
	True positives: 8378
	False positives: 40015
	True negatives: 49954658
	False negatives: 327
	Total comparisons: 48393
---


In [ ]:
data.entities_d1.values.tolist()[0]